In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-23 10:48:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-09-23 10:48:40 (10.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [28]:
import os

for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/Tez Dosyaları/part 3"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '/content/drive/MyDrive/Tez Dosyaları/part 3'.
There are 2 directories and 0 images in '/content/drive/MyDrive/Tez Dosyaları/part 3/Train'.
There are 0 directories and 222 images in '/content/drive/MyDrive/Tez Dosyaları/part 3/Train/Malignant'.
There are 0 directories and 70 images in '/content/drive/MyDrive/Tez Dosyaları/part 3/Train/Benign'.
There are 2 directories and 0 images in '/content/drive/MyDrive/Tez Dosyaları/part 3/Test'.
There are 0 directories and 56 images in '/content/drive/MyDrive/Tez Dosyaları/part 3/Test/Malignant'.
There are 0 directories and 18 images in '/content/drive/MyDrive/Tez Dosyaları/part 3/Test/Benign'.


In [29]:
train_dir = "/content/drive/MyDrive/Tez Dosyaları/part 3/Train"
test_dir = "/content/drive/MyDrive/Tez Dosyaları/part 3/Test"

In [30]:
import tensorflow as tf
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_data= tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                          label_mode="binary",
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                          label_mode="binary",
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMG_SIZE)

Found 292 files belonging to 2 classes.
Found 74 files belonging to 2 classes.


In [31]:

train_data.class_names

['Benign', 'Malignant']

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import random
import math
import csv
import cv2
import os

In [33]:
# 1.  create base model with tf.keras.applications
base_model = tf.keras.applications.VGG19(include_top=False)

# 2. Freeze the base model
base_model.trainable = False

# 3. create inputs into our model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. if using ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

# 7. create the output activation layer
outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)

# 8. compile the model
model_0.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# 9. fit the model
history_0 = model_0.fit(train_data,
                        epochs=40,
                        steps_per_epoch=len(train_data),
                        validation_data=None,  # validation'ı kapat
                        callbacks=[create_tensorboard_callback(dir_name="transfer_learning",
                                                              experiment_name="10_percent_feature_extraction")])

Shape after passing inputs through base model: (None, 7, 7, 512)
Shape after GlobalAveragePooling2D: (None, 512)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extraction/20240923-112659
Epoch 1/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.7275 - loss: 0.9923
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/40


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 178ms/step - accuracy: 0.5484 - loss: 0.8267
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 178ms/step - accuracy: 0.7466 - loss: 0.7192
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.7477 - loss: 0.6161
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step - accuracy: 0.6769 - loss: 0.6779
Epoch 10/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - accuracy: 0.7722 - loss: 0.5918
Epoch 12/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - accuracy: 0.7332 - loss: 0.5767
Epoch 14/40
10/10 ━━━

In [34]:

loss, accuracy = model_0.evaluate(test_data)
print(f"Loss: {loss:.2f}")
print(f"Accuracy: {accuracy:.2f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.6766 - loss: 0.8665
Loss: 0.81
Accuracy: 0.69


In [35]:

# Get the predicted probabilities for the test set
y_pred_probs = model_0.predict(test_data)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_probs > 0.5).astype(int)

# Get the true labels for the test set
y_true = np.concatenate([y for x, y in test_data], axis=0)

# Calculate precision
from sklearn.metrics import precision_score
precision = precision_score(y_true, y_pred)
print("Precision:", precision)


2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 314ms/step
Precision: 0.7536231884057971


In [36]:
from sklearn.metrics import recall_score

# Calculate recall
recall = recall_score(y_true, y_pred)
print("Recall:", recall)

Recall: 0.9285714285714286


In [37]:

from sklearn.metrics import f1_score
# Calculate F1-score
f1 = f1_score(y_true, y_pred)
print("F1-score:", f1)


F1-score: 0.8320000000000001
